# Metro-level home health fragmentation metrics

This notebook joins the POS file (Q4 2025) with the 2023 HHA cost report on CCN, filters to home health (provider type 3), and computes metro-level metrics. Metros are defined by `cbsa_cd` in the POS file. Field definitions: see **pos_data_dictionary.xlsx**.

**Output:** `metro_metrics` table (intermediate) with:
- % of metro revenue by agency size bucket
- % of revenue belonging to top 2 players in the metro
- Number of agencies with $1M–$5M revenue
- % of agencies with ownership change in last 5 years (and % of revenue from those agencies)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

PROJECT_DIR = Path(".").resolve()
POS_PATH = PROJECT_DIR / "pos_file_q4_2025.csv"
COST_REPORT_23_PATH = PROJECT_DIR / "2023" / "CostReporthha_Final_23.csv"
CBSA_CENSUS_PATH = PROJECT_DIR / "cbsa_census.xlsx"
OUTPUT_PATH = PROJECT_DIR / "metro_metrics.csv"

/Users/blakearnold/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 1. Load POS and cost report; join on CCN; filter to home health

In [2]:
# POS: provider type 3 = home health (per CMS POS data)
pos = pd.read_csv(POS_PATH, dtype=str, low_memory=False)
pos["prvdr_num"] = pos["prvdr_num"].astype(str).str.strip()
pos = pos[pos["prvdr_type_id"] == "3"]

# Metro = cbsa_cd; drop rows with missing or non-numeric cbsa
pos["cbsa_cd"] = pos["cbsa_cd"].astype(str).str.strip()
pos = pos[pos["cbsa_cd"].str.match(r"^\d+$", na=False)]

print(f"POS home health (prvdr_type_id=3) with valid cbsa_cd: {len(pos):,}")

POS home health (prvdr_type_id=3) with valid cbsa_cd: 23,146


In [3]:
# Cost report 2023: CCN and total revenue
cost = pd.read_csv(COST_REPORT_23_PATH, dtype=str, low_memory=False)
cost["Provider CCN"] = cost["Provider CCN"].astype(str).str.strip()
rev_col = "Gross Patient Revenues Total"
cost[rev_col] = pd.to_numeric(cost[rev_col], errors="coerce")
cost = cost[["Provider CCN", rev_col]].dropna(subset=[rev_col])
cost = cost[cost[rev_col] >= 0]

print(f"Cost report 2023 rows with revenue: {len(cost):,}")

Cost report 2023 rows with revenue: 7,220


In [4]:
# Join POS (home health) with cost report on CCN; include chow_dt for ownership-change metrics
pos_cols = ["prvdr_num", "cbsa_cd", "fac_name", "state_cd", "chow_dt"]
df = pos[pos_cols].merge(
    cost,
    left_on="prvdr_num",
    right_on="Provider CCN",
    how="inner"
)
# One row per CCN (POS may have one per CCN; cost report one per CCN)
df = df.drop_duplicates(subset=["prvdr_num"])
df = df.rename(columns={"Gross Patient Revenues Total": "revenue"})

print(f"Joined rows (home health with 2023 cost report revenue): {len(df):,}")
print(f"Metros (cbsa_cd): {df['cbsa_cd'].nunique()}")

Joined rows (home health with 2023 cost report revenue): 5,880
Metros (cbsa_cd): 401


## 2. Agency size buckets and revenue-share by metro

In [5]:
BUCKETS = [
    (0, 1e6, "<$1M"),
    (1e6, 2e6, "$1-2M"),
    (2e6, 5e6, "$2-5M"),
    (5e6, 10e6, "$5-10M"),
    (10e6, 20e6, "$10-20M"),
    (20e6, 50e6, "$20-50M"),
    (50e6, np.inf, "$50M+"),
]

def revenue_bucket(revenue: float) -> str:
    for lo, hi, label in BUCKETS:
        if lo <= revenue < hi:
            return label
    return "$50M+"

df["revenue_bucket"] = df["revenue"].map(revenue_bucket)

In [6]:
# Metro-level: total revenue and revenue by bucket
metro_total_rev = df.groupby("cbsa_cd")["revenue"].sum().to_dict()
df["metro_revenue"] = df["cbsa_cd"].map(metro_total_rev)

# Revenue by bucket within metro (for pct)
bucket_rev = df.groupby(["cbsa_cd", "revenue_bucket"])["revenue"].sum().unstack(fill_value=0)
bucket_labels = [b[2] for b in BUCKETS]
for lb in bucket_labels:
    if lb not in bucket_rev.columns:
        bucket_rev[lb] = 0
bucket_rev = bucket_rev.reindex(columns=bucket_labels, fill_value=0)
pct_by_bucket = bucket_rev.div(bucket_rev.sum(axis=1), axis=0) * 100

## 3. Top 2 players share and count of agencies $1M–$5M

In [7]:
# Per metro: rank agencies by revenue, sum top 2 (return Series via groupby on revenue)
def top2_revenue_share(rev: pd.Series) -> float:
    total = rev.sum()
    if total <= 0:
        return np.nan
    return 100 * rev.nlargest(2).sum() / total

top2_share = df.groupby("cbsa_cd")["revenue"].apply(top2_revenue_share)
top2_share.name = "pct_revenue_top2"

# Count agencies with revenue in [$1M, $5M] per metro
df_1m_5m = df[(df["revenue"] >= 1e6) & (df["revenue"] < 5e6)]
count_1m_5m = df_1m_5m.groupby("cbsa_cd").size().reindex(df["cbsa_cd"].unique()).fillna(0).astype(int)
count_1m_5m.name = "n_agencies_1m_to_5m"

## 4. Ownership change in last 5 years (using POS `chow_dt`)

**chow_dt** in the POS file is the last date ownership changed (CHOW = Change of Ownership). We flag an agency as having an ownership change in the last 5 years if `chow_dt` is present and falls within 5 years before the reference date (end of Q4 2025).

In [8]:
# chow_dt = last date ownership changed (POS). Reference = end of Q4 2025; "last 5 years" = on or after 2021-01-01
REFERENCE_DATE = pd.Timestamp("2025-12-31")
CUTOFF_DATE = REFERENCE_DATE - pd.DateOffset(years=5)  # 2020-12-31

chow = df["chow_dt"].fillna("").astype(str).str.strip()
chow = chow.replace(["Not Available", "Not Applicable", ""], np.nan)
df["chow_date"] = pd.to_datetime(chow, errors="coerce")
df["ownership_change"] = df["chow_date"].notna() & (df["chow_date"] >= CUTOFF_DATE)

print(f"Agencies with ownership change in last 5 years (chow_dt >= {CUTOFF_DATE.date()}): {df['ownership_change'].sum():,}")

Agencies with ownership change in last 5 years (chow_dt >= 2020-12-31): 68


In [9]:
# Metro: % of agencies with ownership change; % of revenue from those agencies
def pct_agencies_ownership_change(g: pd.DataFrame) -> float:
    n = len(g)
    if n == 0:
        return np.nan
    return 100 * g["ownership_change"].sum() / n

def pct_revenue_ownership_change(g: pd.DataFrame) -> float:
    total = g["revenue"].sum()
    if total <= 0:
        return np.nan
    rev_changed = g.loc[g["ownership_change"], "revenue"].sum()
    return 100 * rev_changed / total

pct_agencies_chow = df.groupby("cbsa_cd").apply(pct_agencies_ownership_change)
pct_agencies_chow.name = "pct_agencies_ownership_change"
pct_rev_chow = df.groupby("cbsa_cd").apply(pct_revenue_ownership_change)
pct_rev_chow.name = "pct_revenue_ownership_change"

/var/folders/h8/qjsv1g155wn2pxvqzdtylswc0000gn/T/ipykernel_46843/3572842036.py:15: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pct_agencies_chow = df.groupby("cbsa_cd").apply(pct_agencies_ownership_change)
/var/folders/h8/qjsv1g155wn2pxvqzdtylswc0000gn/T/ipykernel_46843/3572842036.py:17: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pct_rev_chow = df.groupby("cbsa_cd").apply(pct_revenue_ownership_change)


## 5. CBSA names from Census file

In [10]:
# Load CBSA code -> name from cbsa_census.xlsx (Census Bureau list; multiple rows per CBSA, same name)
cbsa_list = pd.read_excel(CBSA_CENSUS_PATH, engine="openpyxl", header=1)
code_col = cbsa_list.columns[0]
title_col = cbsa_list.columns[3]  # CBSA Title
cbsa_names = (
    cbsa_list[[code_col, title_col]]
    .rename(columns={code_col: "cbsa_cd", title_col: "cbsa_name"})
    .astype(str)
    .apply(lambda x: x.str.strip())
)
cbsa_names["cbsa_cd"] = cbsa_names["cbsa_cd"].replace("nan", "")
cbsa_names = cbsa_names[cbsa_names["cbsa_cd"].str.match(r"^\d+$", na=False)].drop_duplicates(subset=["cbsa_cd"])
print(f"Loaded {len(cbsa_names):,} unique CBSA names from cbsa_census.xlsx")

Loaded 935 unique CBSA names from cbsa_census.xlsx


## 6. Build metro_metrics table and save

In [11]:
# Metro-level totals
metro_n_agencies = df.groupby("cbsa_cd").size()
metro_n_agencies.name = "n_agencies"
metro_revenue_tot = df.groupby("cbsa_cd")["revenue"].sum()
metro_revenue_tot.name = "total_revenue"

# Assemble metro_metrics: one row per metro (index = cbsa_cd from pct_by_bucket)
metro_metrics = pd.DataFrame(index=pct_by_bucket.index)
metro_metrics["n_agencies"] = metro_n_agencies
metro_metrics["total_revenue"] = metro_revenue_tot
metro_metrics["pct_revenue_under_1M"] = pct_by_bucket["<$1M"]
metro_metrics["pct_revenue_1M_2M"] = pct_by_bucket["$1-2M"]
metro_metrics["pct_revenue_2M_5M"] = pct_by_bucket["$2-5M"]
metro_metrics["pct_revenue_5M_10M"] = pct_by_bucket["$5-10M"]
metro_metrics["pct_revenue_10M_20M"] = pct_by_bucket["$10-20M"]
metro_metrics["pct_revenue_20M_50M"] = pct_by_bucket["$20-50M"]
metro_metrics["pct_revenue_50M_plus"] = pct_by_bucket["$50M+"]
# Reindex to metro index; ensure 1D for assignment (groupby.apply can return Series or DataFrame)
def _to_series(s, idx):
    r = s.reindex(idx)
    return r.iloc[:, 0] if isinstance(r, pd.DataFrame) else r
metro_metrics["pct_revenue_top2"] = _to_series(top2_share, metro_metrics.index)
metro_metrics["n_agencies_1m_to_5m"] = count_1m_5m.reindex(metro_metrics.index).fillna(0).astype(int)
metro_metrics["pct_agencies_ownership_change"] = _to_series(pct_agencies_chow, metro_metrics.index)
metro_metrics["pct_revenue_ownership_change"] = _to_series(pct_rev_chow, metro_metrics.index)

metro_metrics.index.name = "cbsa_cd"
metro_metrics = metro_metrics.reset_index()
# Add CBSA names from Census file (left merge; missing names stay NaN)
metro_metrics = metro_metrics.merge(cbsa_names, on="cbsa_cd", how="left")
# Order columns: cbsa_cd, cbsa_name, then the rest
cols = ["cbsa_cd", "cbsa_name"] + [c for c in metro_metrics.columns if c not in ("cbsa_cd", "cbsa_name")]
metro_metrics = metro_metrics[cols]
metro_metrics.to_csv(OUTPUT_PATH, index=False)
print(f"Saved metro_metrics to {OUTPUT_PATH}")
print(metro_metrics.head(10))

Saved metro_metrics to /Users/blakearnold/Documents/home_health_targets/metro_metrics.csv
  cbsa_cd                          cbsa_name  n_agencies  total_revenue  \
0   10180                        Abilene, TX           7     58445275.0   
1   10380                      Aguadilla, PR           1      4760963.0   
2   10420                          Akron, OH           6     64715137.0   
3   10500                         Albany, GA           2      3897163.0   
4   10580        Albany-Schenectady-Troy, NY           4     85174966.0   
5   10740                    Albuquerque, NM          15     75934676.0   
6   10780                     Alexandria, LA           8     22460535.0   
7   10900  Allentown-Bethlehem-Easton, PA-NJ           9     67149915.0   
8   11020                        Altoona, PA           5     73485926.0   
9   11100                       Amarillo, TX           9     50484030.0   

   pct_revenue_under_1M  pct_revenue_1M_2M  pct_revenue_2M_5M  \
0              0.00

In [12]:
# Preview column names and shapes
print("Columns:", list(metro_metrics.columns))
print("Shape:", metro_metrics.shape)

Columns: ['cbsa_cd', 'cbsa_name', 'n_agencies', 'total_revenue', 'pct_revenue_under_1M', 'pct_revenue_1M_2M', 'pct_revenue_2M_5M', 'pct_revenue_5M_10M', 'pct_revenue_10M_20M', 'pct_revenue_20M_50M', 'pct_revenue_50M_plus', 'pct_revenue_top2', 'n_agencies_1m_to_5m', 'pct_agencies_ownership_change', 'pct_revenue_ownership_change']
Shape: (401, 15)
